In [2]:
#this converts lon lat coordinates into utm koordinates
from pyproj import Transformer

# Initialisieren der Transformer
wgs_to_utm_transformer = Transformer.from_crs("epsg:4326", "epsg:25832", always_xy=True)
utm_to_wgs_transformer = Transformer.from_crs("epsg:25832", "epsg:4326", always_xy=True)

# Verwenden der definierten Funktion, um die Transformation für die angegebenen Koordinaten durchzuführen:
# 48,131507 (Breitengrad), 11,689532 (Längengrad)

# Definierte Funktion zur Transformation
def transform_lat_lon_to_utm(lat, lon):
    """
    Transformiert geografische Koordinaten in ETRS89 / UTM Zone 32N Koordinaten.
    
    :param lat: Breitengrad
    :param lon: Längengrad
    :return: Tuple (x, y) repräsentiert die transformierten UTM Koordinaten (Easting, Northing)
    """
    transformer = Transformer.from_crs("epsg:4326", "epsg:25832", always_xy=True)
    x, y = transformer.transform(lon, lat)
    return x, y

# Koordinaten für die Transformation
lat_input, lon_input = 48.131507, 11.689532

# Durchführung der Transformation
transform_lat_lon_to_utm(lat_input, lon_input)

ModuleNotFoundError: No module named 'pyproj'

In [3]:
#this checks the geojson file to map the coordinates to the stadtbezirk
import geopandas as gpd
from shapely.geometry import Point

def check_location_inside(file_path, location):
    # Load the file containing the Bezirks polygons
    bezirks = gpd.read_file(file_path)
    
    # Convert the location to a Point geometry
    point = Point(location)
    
    # Check if the point is inside any of the Bezirks polygons
    for index, row in bezirks.iterrows():
        if point.within(row['geometry']):
            return row['sb_nummer']  # Return the ID of the Bezirk if the location is inside
    
    return None  # Return None if the location is not inside any Bezirk

# Example usage:
file_path = "/Users/janis/Desktop/Muc_electro_vehicles/data/muc_geojson.geojson"  # Replace with the actual path to your GeoJSON file
location = (700108.6343251554, 5334415.579024226)
bezirk_id = check_location_inside(file_path, location)
if bezirk_id is not None:
    print(f"The location is inside Bezirk {bezirk_id}.")
else:
    print("The location is not inside any Bezirk.")

ModuleNotFoundError: No module named 'geopandas'

In [3]:
### this code is on function that converts lon lat input to stadbezirk muc output

from pyproj import Transformer
import geopandas as gpd
from shapely.geometry import Point

# Define the transformer for converting latitude and longitude to UTM
transformer = Transformer.from_crs("epsg:4326", "epsg:25832", always_xy=True)

def transform_lat_lon_to_utm(lat, lon):
    """
    Transform latitude and longitude to ETRS89 / UTM Zone 32N coordinates.

    :param lat: Latitude
    :param lon: Longitude
    :return: Tuple (x, y) representing the transformed UTM coordinates (Easting, Northing)
    """
    x, y = transformer.transform(lon, lat)
    return x, y

def check_location_inside(file_path, lat, lon):
    """
    Checks if the given geographic location (latitude and longitude) is inside any Stadtbezirk.

    :param file_path: Path to the GeoJSON file containing Stadtbezirk polygons
    :param lat: Latitude of the location
    :param lon: Longitude of the location
    :return: The ID of the Stadtbezirk if inside, otherwise None
    """
    # Load the GeoJSON file with Stadtbezirk polygons
    bezirks = gpd.read_file(file_path)

    # Transform latitude and longitude to UTM coordinates
    x, y = transform_lat_lon_to_utm(lat, lon)

    # Convert the UTM coordinates to a Point geometry
    point = Point(x, y)

    # Check if the point is inside any of the Stadtbezirk polygons
    for _, row in bezirks.iterrows():
        if point.within(row['geometry']):
            return row['sb_nummer']  # Return the ID of the Stadtbezirk

    return None  # Return None if the location is not inside any Stadtbezirk

# Example usage:
file_path = "../data/muc_geojson.geojson"  # Update with the correct path
lat, lon = 48.131507, 11.689532  # Sample coordinates
bezirk_id = check_location_inside(file_path, lat, lon)
if bezirk_id:
    print(f"The location is inside Bezirk {bezirk_id}.")
else:
    print("Not in MUC.")


ModuleNotFoundError: No module named 'pyproj'

In [3]:
import pandas as pd

# Load the CSV file into a DataFrame
file_path_csv = '../data/ladestationen_muc_janis.csv'
df = pd.read_csv(file_path_csv)

# Replace ',' with '.' in the 'Längengrad' and 'Breitengrad' columns and convert them to floats
df['Längengrad'] = df['Längengrad'].str.replace(',', '.').astype(float)
df['Breitengrad'] = df['Breitengrad'].str.replace(',', '.').astype(float)

# Define the path to the GeoJSON file
geojson_file_path = '../data/muc_geojson.geojson'  # Update this path

# Apply the transformation and check function to each row in the DataFrame, create a new column "stadtbezirk" with the returned value of the mapping
df['Stadtbezirk'] = df.apply(lambda row: check_location_inside(geojson_file_path, row['Breitengrad'], row['Längengrad']), axis=1)

# Save the modified DataFrame to a new CSV file
output_file_path = 'Ladesaeulenregister_with_Stadtbezirk.csv'
df.to_csv(output_file_path, index=False)


NameError: name 'check_location_inside' is not defined

In [4]:
#show data on map

import plotly.graph_objects as go
import json
import pandas as pd

df = pd.read_csv("../data/df_BNA_merge.csv")
df['Bezirk'] = df['Bezirk'].astype(str)



with open("../data/muc_lon_lat_angepasst.geojson", 'r') as file:
    geojson_muc = json.load(file)



import pandas as pd
#df = px.data.election()

# Sample Choropleth mapbox using Plotly GO
import plotly.graph_objects as go

fig = go.Figure(go.Choroplethmapbox(geojson=geojson_muc, locations=df["Bezirksname"], z=df["Punkte_tsd_Einw"],featureidkey="properties.name",
                                    colorscale="Viridis", zmin=0, zmax=5,
                                    marker_opacity=0.5, marker_line_width=1))
fig.update_layout(mapbox_style="carto-positron",
                  mapbox_zoom=9,mapbox_center = {"lat": 48.13843756611454, "lon": 11.576095494287193} )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})


#muc is: 48.13843756611454, 11.576095494287193

fig.show()



FileNotFoundError: [Errno 2] No such file or directory: '../data/muc_lon_lat_angepasst.geojson'

In [4]:
import json
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

# Assuming the file paths are correct and accessible
df = pd.read_csv("../data/Ladesaulenregister_with_Stadtbezirk_B.csv")

# Correcting the field names if they are in the wrong order:
# Ensure 'Längengrad' is used for longitude (x-axis/horizontal) and 'Breitengrad' for latitude (y-axis/vertical).
df['Längengrad'] = df['Längengrad'].str.replace(',', '.').astype(float)
df['Breitengrad'] = df['Breitengrad'].str.replace(',', '.').astype(float)

# Load GeoJSON for Munich if needed for additional context or layers, not used directly in this snippet
with open("../data/muc_lon_lat_angepasst.geojson", 'r') as file:
    geojson_muc = json.load(file)

# Create a figure with charging points
fig_all_charging_points = go.Figure(go.Scattermapbox(
    lon = df['Längengrad'],  # X-axis should correspond to longitude
    lat = df['Breitengrad'],  # Y-axis should correspond to latitude
    mode = 'markers',
    marker = go.scattermapbox.Marker(size=7),
))

# Update layout to focus on the points, with an appropriate zoom level and center
fig_all_charging_points.update_layout(
    mapbox_style="light",  # Or "open-street-map", "dark", etc.
    mapbox=dict(
        bearing=0,
        center=go.layout.mapbox.Center(
            lat=df['Breitengrad'].mean(),
            lon=df['Längengrad'].mean()
        ),
        pitch=0,
        zoom=10  # Adjust zoom level as needed to best fit your data points
    ),
    showlegend=False
)

fig_all_charging_points.update_layout(mapbox_style="carto-positron",
                  mapbox_zoom=9,mapbox_center = {"lat": 48.13843756611454, "lon": 11.576095494287193} )
fig_all_charging_points.update_layout(margin={"r":0,"t":0,"l":0,"b":0})




fig_all_charging_points.show()



FileNotFoundError: [Errno 2] No such file or directory: '../data/muc_lon_lat_angepasst.geojson'

In [6]:
#just to convert - not necessary

import pandas as pd
import numpy as np

df1 = pd.read_csv("/Users/janis/Desktop/Muc_electro_vehicles/data/ladestationen_muc_janis.csv")
df2 = pd.read_csv("../data/Ladesaeulenregister_mit_Bezirk.csv")

print(len(df1))
print(len(df2))

df1['Stadtbezirk'] = df2['Stadtbezirk']
df1['Stadtbezirk'] = df1['Stadtbezirk'].apply(lambda x: int(x) if pd.notnull(x) else x)


# Display the modified dataframe to verify the changes.
#df1



# Step 3: Save the updated df1 to a CSV file
df1.to_csv('updated_dataframe.csv', index=False)


FileNotFoundError: [Errno 2] No such file or directory: '/Users/janis/Desktop/Muc_electro_vehicles/data/ladestationen_muc_janis.csv'

In [96]:
#show all charging points with district boarders
import json
import pandas as pd

# Load your main dataset and ensure 'Bezirk' is in the correct data type
df_main = pd.read_csv("../data/df_BNA_merge.csv")
df_main['Bezirk'] = df_main['Bezirk'].astype(str)

# Load your charging stations data, correcting the longitude and latitude data types
df = pd.read_csv("../data/Ladesaulenregister_with_Stadtbezirk_B.csv")
df['Längengrad'] = df['Längengrad'].str.replace(',', '.').astype(float)
df['Breitengrad'] = df['Breitengrad'].str.replace(',', '.').astype(float)

# Load the Munich GeoJSON file
with open("../data/muc_lon_lat_angepasst.geojson", 'r') as file:
    geojson_muc = json.load(file)

# Initialize your figure
fig_all_charging_points = go.Figure()

# Add the scatter plot layer for charging stations
fig_all_charging_points.add_trace(go.Scattermapbox(
    lon=df['Längengrad'],
    lat=df['Breitengrad'],
    mode='markers',
    marker=go.scattermapbox.Marker(size=7),
    name='Charging Stations'
))

# Add district borders without fill using a Line trace instead of Choroplethmapbox
for feature in geojson_muc['features']:
    feature_coordinates = feature['geometry']['coordinates']
    # Check and adapt to different geometry types
    if feature['geometry']['type'] == 'Polygon':
        for coords in feature_coordinates:
            # Extracting longitude and latitude for the Line trace
            lon, lat = zip(*coords)
            fig_all_charging_points.add_trace(go.Scattermapbox(
                lon=lon,
                lat=lat,
                mode='lines',
                line=go.scattermapbox.Line(color='black', width=1),
                showlegend=False
            ))
    elif feature['geometry']['type'] == 'MultiPolygon':
        for part in feature_coordinates:
            for coords in part:
                lon, lat = zip(*coords)
                fig_all_charging_points.add_trace(go.Scattermapbox(
                    lon=lon,
                    lat=lat,
                    mode='lines',
                    line=go.scattermapbox.Line(color='black', width=2),
                    showlegend=False
                ))

# Update the layout for the combined map visualization
fig_all_charging_points.update_layout(
    mapbox_style="carto-positron",
    mapbox=dict(
        bearing=0,
        center=dict(
            lat=df['Breitengrad'].mean(),
            lon=df['Längengrad'].mean()
        ),
        pitch=0,
        zoom=9
    ),
    margin={"r": 0, "t": 0, "l": 0, "b": 0}
)

# Show the figure
fig_all_charging_points.show()



ValueError: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido
